# Polynomial Transformer Testing Suite

This notebook tests various components of the Polynomial Transformer implementation.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import sys
sys.path.append('/home/ashishkangen/Projects/polyformer')

from former import (
    TransformerBlock, 
    PolyTransformerBlock,
    GTransformer,
    PolyGTransformer,
    CTransformer,
    PolyCTransformer
)

from former.util import (
    CP, CP_sparse_LU, CP_sparse_degree, CP_sparse_degree_LU
)

from former.util import *
from former.util.poly_utils import *

## Utility Functions for Testing

In [2]:
def test_layer(layer, input_shape, verbose=True):
    """Test if a layer can process input and produce output without errors."""
    with torch.no_grad():  # No need for gradient tracking during testing
        x = torch.randn(input_shape)
        try:
            out = layer(x)
            if verbose:
                print(f"✅ Layer works! Input shape: {input_shape}, Output shape: {out.shape}")
            return out.shape
        except Exception as e:
            print(f"❌ Layer failed with input shape {input_shape}. Error: {e}")
            return None

def count_parameters(model):
    """Count the total number of trainable parameters in a model."""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def parameter_comparison(model1, model2, names=['Model 1', 'Model 2']):
    """Compare parameter counts between two models."""
    params1 = count_parameters(model1)
    params2 = count_parameters(model2)
    
    print(f"{names[0]} parameters: {params1:,}")
    print(f"{names[1]} parameters: {params2:,}")
    
    if params1 > params2:
        print(f"{names[0]} has {params1-params2:,} more parameters ({params1/params2:.2f}x)")
    else:
        print(f"{names[1]} has {params2-params1:,} more parameters ({params2/params1:.2f}x)")

## 1. Testing the PolyTransformerBlock

In [4]:
# Configuration parameters
emb = 64          # Embedding dimension
heads = 4         # Number of attention heads
seq_length = 32   # Sequence length
batch_size = 8    # Batch size

# Create a standard transformer block
transformer_block = TransformerBlock(
    emb=emb,
    heads=heads,
    mask=True, 
    seq_length=seq_length
)

# Create a polynomial transformer block
poly_block = PolyTransformerBlock(
    emb=emb,
    heads=heads,
    mask=True, 
    seq_length=seq_length,
    degree=2,
    poly_class=CP,
    use_relu=True,
    ff_hidden_mult=2,
    #attention_type='sparse_inplace'
)

# Test both blocks with the same input
input_shape = (batch_size, seq_length, emb)
print("Testing standard TransformerBlock:")

with torch.no_grad():  # No need for gradient tracking during testing
    x = torch.randn(input_shape)
    out = poly_block(x)

std_output_shape = test_layer(transformer_block, input_shape)

print("\nTesting PolyTransformerBlock:")
poly_output_shape = test_layer(poly_block, input_shape)

# Verify that both blocks produce the same output shape
if std_output_shape == poly_output_shape:
    print("\n✅ Both blocks produce outputs of the same shape!")
else:
    print(f"\n❌ Output shapes differ! Standard: {std_output_shape}, Poly: {poly_output_shape}")

# Compare parameter counts
print("\nParameter comparison:")
parameter_comparison(transformer_block, poly_block, ["Standard TransformerBlock", "PolyTransformerBlock"])

Hidden dimension: 64
Testing standard TransformerBlock:


RuntimeError: The size of tensor a (256) must match the size of tensor b (32) at non-singleton dimension 1

## 2. Testing Different Polynomial Classes

In [5]:
# Test various polynomial network implementations
poly_classes = [CP, CP_sparse_LU, CP_sparse_degree, CP_sparse_degree_LU]
poly_class_names = ['CP', 'CP_sparse_LU', 'CP_sparse_degree', 'CP_sparse_degree_LU']

# Test with different degrees
degrees = [1, 2, 3]

results = {}

for degree in degrees:
    print(f"\n\nTesting with degree = {degree}:")
    
    for cls, name in zip(poly_classes, poly_class_names):
        print(f"\nTesting {name}:")
        
        try:
            block = PolyTransformerBlock(
                emb=emb,
                heads=heads,
                mask=True, 
                seq_length=seq_length,
                degree=degree,
                poly_class=cls
            )
            
            _ = test_layer(block, input_shape, verbose=False)
            param_count = count_parameters(block)
            results[(degree, name)] = param_count
            
            print(f"✅ {name} with degree {degree} works! Parameters: {param_count:,}")
            
        except Exception as e:
            print(f"❌ {name} with degree {degree} failed. Error: {e}")
            results[(degree, name)] = None

# Print summary table
print("\n\nSummary of parameter counts:")
print("-" * 60)
print(f"{'Degree':<10} {'Class':<20} {'Parameters':<15}")
print("-" * 60)

for (degree, cls_name), params in sorted(results.items()):
    if params is not None:
        print(f"{degree:<10} {cls_name:<20} {params:,}")
    else:
        print(f"{degree:<10} {cls_name:<20} {'Failed'}")



Testing with degree = 1:

Testing CP:
❌ Layer failed with input shape (8, 32, 64). Error: The size of tensor a (256) must match the size of tensor b (32) at non-singleton dimension 1
✅ CP with degree 1 works! Parameters: 119,424

Testing CP_sparse_LU:
❌ Layer failed with input shape (8, 32, 64). Error: The size of tensor a (256) must match the size of tensor b (32) at non-singleton dimension 1
✅ CP_sparse_LU with degree 1 works! Parameters: 119,424

Testing CP_sparse_degree:
❌ Layer failed with input shape (8, 32, 64). Error: The size of tensor a (256) must match the size of tensor b (32) at non-singleton dimension 1
✅ CP_sparse_degree with degree 1 works! Parameters: 119,424

Testing CP_sparse_degree_LU:
❌ Layer failed with input shape (8, 32, 64). Error: The size of tensor a (256) must match the size of tensor b (32) at non-singleton dimension 1
✅ CP_sparse_degree_LU with degree 1 works! Parameters: 119,424


Testing with degree = 2:

Testing CP:
❌ Layer failed with input shape (8,

## 3. Testing Full Transformer Models

In [6]:
# Configuration for full transformer models
emb = 128  # Embedding dimension
heads = 8  # Number of attention heads
depth = 3  # Number of transformer blocks
seq_length = 64  # Sequence length
num_tokens = 10000  # Vocabulary size
num_classes = 5  # For classification model

print("1. Testing generative models (GTransformer vs PolyGTransformer):")
g_transformer = GTransformer(emb, heads, depth, seq_length, num_tokens)
poly_g_transformer = PolyGTransformer(emb, heads, depth, seq_length, num_tokens, degree=2)

# Test with a batch of token indices
token_indices = torch.randint(0, num_tokens, (4, seq_length))

with torch.no_grad():
    g_out = g_transformer(token_indices)
    poly_g_out = poly_g_transformer(token_indices)
    
    print(f"GTransformer output shape: {g_out.shape}")
    print(f"PolyGTransformer output shape: {poly_g_out.shape}")
    
    if g_out.shape == poly_g_out.shape:
        print("✅ Both generative models produce the same output shape!")
    else:
        print("❌ Generative models produce different output shapes!")

print("\nParameter comparison:")
parameter_comparison(g_transformer, poly_g_transformer, ["GTransformer", "PolyGTransformer"])

print("\n2. Testing classification models (CTransformer vs PolyCTransformer):")
c_transformer = CTransformer(emb, heads, depth, seq_length, num_tokens, num_classes)
poly_c_transformer = PolyCTransformer(emb, heads, depth, seq_length, num_tokens, num_classes, degree=2)

with torch.no_grad():
    c_out = c_transformer(token_indices)
    poly_c_out = poly_c_transformer(token_indices)
    
    print(f"CTransformer output shape: {c_out.shape}")
    print(f"PolyCTransformer output shape: {poly_c_out.shape}")
    
    if c_out.shape == poly_c_out.shape:
        print("✅ Both classification models produce the same output shape!")
    else:
        print("❌ Classification models produce different output shapes!")

print("\nParameter comparison:")
parameter_comparison(c_transformer, poly_c_transformer, ["CTransformer", "PolyCTransformer"])

1. Testing generative models (GTransformer vs PolyGTransformer):


RuntimeError: Expected object of backend CPU but got backend CUDA for argument #3 'index'

## 4. Testing Different Attention Types with Polynomial Networks

In [ ]:
attention_types = [
    'default', 'alt', 'wide', 'gpt2', 'narrow', 'sparse_graph'
]

for att_type in attention_types:
    print(f"\nTesting PolyTransformerBlock with attention_type='{att_type}'")
    
    try:
        # Some attention types might need special handling
        kwargs = {}
        if att_type == 'relative':
            # Relative attention needs position embeddings
            pos_emb = nn.Embedding(seq_length * 2 - 1, emb)
            kwargs['pos_embedding'] = pos_emb
            
        block = PolyTransformerBlock(
            emb=emb,
            heads=heads,
            mask=True, 
            seq_length=seq_length,
            degree=2,
            attention_type=att_type,
            **kwargs
        )
        
        out_shape = test_layer(block, input_shape)
        if out_shape is not None:
            print(f"  ✅ PolyTransformerBlock with {att_type} attention works!")
        
    except Exception as e:
        print(f"  ❌ PolyTransformerBlock with {att_type} attention failed. Error: {e}")

## 5. Test Training with Small Synthetic Task

In [ ]:
import torch.optim as optim

def train_for_epochs(model, epochs=5, batch_size=32, seq_len=16, vocab_size=100):
    """Train a model for a few epochs on a simple synthetic sequence task."""
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Simple synthetic task: predict next token in a sequence
    losses = []
    
    for epoch in range(epochs):
        running_loss = 0.0
        
        # Generate synthetic data - sequences where each token is the previous token + 1
        for i in range(5):
            # Generate random starting point for sequences
            starts = torch.randint(0, vocab_size - seq_len, (batch_size, 1))
            # Create sequences with a pattern (each position is previous + 1)
            seqs = torch.cat([starts + j for j in range(seq_len)], dim=1)
            
            # Inputs are all but last token, targets are all but first token
            inputs = seqs[:, :-1]
            targets = seqs[:, 1:].contiguous().view(-1)
            
            # Forward pass
            outputs = model(inputs)
            
            # Reshape for loss calculation
            outputs = outputs.view(-1, vocab_size)
            
            # Calculate loss
            loss = criterion(outputs, targets)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        avg_loss = running_loss / 5
        losses.append(avg_loss)
        print(f'Epoch {epoch+1}, Loss: {avg_loss:.4f}')
    
    return losses

# Setup simplified models for training test
vocab_size = 100
seq_len = 16
emb = 32
heads = 2
depth = 2

print("Training GTransformer:")
g_model = GTransformer(emb, heads, depth, seq_len, vocab_size)
g_losses = train_for_epochs(g_model, epochs=3, vocab_size=vocab_size, seq_len=seq_len)

print("\nTraining PolyGTransformer:")
poly_g_model = PolyGTransformer(emb, heads, depth, seq_len, vocab_size, degree=2)
poly_g_losses = train_for_epochs(poly_g_model, epochs=3, vocab_size=vocab_size, seq_len=seq_len)

# Plot loss comparison
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(g_losses, label='GTransformer')
plt.plot(poly_g_losses, label='PolyGTransformer')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Comparison')
plt.legend()
plt.grid(True)
plt.show()

## 6. Test Inference Speed Comparison

In [ ]:
import time

def measure_inference_time(model, input_tensor, runs=10):
    """Measure average inference time over multiple runs."""
    # Warm-up run
    with torch.no_grad():
        _ = model(input_tensor)
    
    # Timed runs
    start_time = time.time()
    
    with torch.no_grad():
        for _ in range(runs):
            _ = model(input_tensor)
            
    end_time = time.time()
    avg_time = (end_time - start_time) / runs
    
    return avg_time

# Setup models for speed comparison
emb = 256
heads = 8
depth = 4
seq_length = 64
vocab_size = 10000
batch_size = 16

# Create input tensor for inference
input_tensor = torch.randint(0, vocab_size, (batch_size, seq_length))

# Create models for comparison
models = {
    "GTransformer": GTransformer(emb, heads, depth, seq_length, vocab_size),
    "PolyGTransformer (degree=1)": PolyGTransformer(emb, heads, depth, seq_length, vocab_size, degree=1),
    "PolyGTransformer (degree=2)": PolyGTransformer(emb, heads, depth, seq_length, vocab_size, degree=2),
    "PolyGTransformer (degree=3)": PolyGTransformer(emb, heads, depth, seq_length, vocab_size, degree=3),
    "PolyGTransformer (CP_sparse_LU)": PolyGTransformer(emb, heads, depth, seq_length, vocab_size, degree=2, poly_class=CP_sparse_LU)
}

# Run timing tests
times = {}
for name, model in models.items():
    print(f"Measuring inference time for {name}...")
    times[name] = measure_inference_time(model, input_tensor)

# Print results
print("\nInference Speed Comparison:")
print("-" * 70)
print(f"{'Model':<30} {'Time (ms)':<10} {'Relative Speed':<15}")
print("-" * 70)

# Use standard transformer as baseline
baseline = times["GTransformer"]
for name, time_taken in times.items():
    rel_speed = baseline / time_taken
    print(f"{name:<30} {time_taken*1000:>9.2f} {rel_speed:>14.2f}x")